In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as ExcelImage
from io import BytesIO

# -------------------------
# Step 1: Load & Preprocess
# -------------------------
df = pd.read_csv('D:/DYD/AI-ML/Python-Scripts/Data/SV_SG.csv')
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

# -------------------------
# Step 2: Configurable Dates
# -------------------------
start_date = '2025-01-01'
end_date = '2025-05-31'
filtered_df = df[(df['order_date'] >= start_date) & (df['order_date'] <= end_date)]

# -------------------------
# Step 3: Create Pivot Table
# -------------------------
pivot_table = pd.pivot_table(
    filtered_df,
    values='Customer_Number',
    index=['SV_NAME', 'selected_time_slot_name'],
    columns='order_date',
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name='Total'
)

output_file = 'D:/DYD/AI-ML/Python-Scripts/Data/SV_SG_Pivot.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    filtered_df.to_excel(writer, sheet_name='Filtered Data', index=False)
    pivot_table.to_excel(writer, sheet_name='Pivot All Status')

    for status in filtered_df['Status'].dropna().unique():
        status_df = filtered_df[filtered_df['Status'] == status]
        status_pivot = pd.pivot_table(
            status_df,
            values='Customer_Number',
            index=['SV_NAME', 'selected_time_slot_name'],
            columns='order_date',
            aggfunc='count',
            fill_value=0,
            margins=True,
            margins_name='Total'
        )
        sheet_name = f"Pivot_{status[:28]}"
        status_pivot.to_excel(writer, sheet_name=sheet_name)

print("✅ Pivot tables written to Excel.")

# -------------------------
# Step 4: Prepare Chart Data
# -------------------------
chart_data = (
    filtered_df.groupby([filtered_df['order_date'].dt.date, 'Status'])
    .agg({'Customer_Number': 'count'})
    .reset_index()
)

pivot_chart = chart_data.pivot(index='order_date', columns='Status', values='Customer_Number').fillna(0)
pivot_chart['Cumulative'] = pivot_chart.sum(axis=1)
pivot_chart = pivot_chart.sort_index()
pivot_chart.reset_index(inplace=True)

# -------------------------
# Step 5: Generate Charts In-Memory
# -------------------------
sns.set(style="whitegrid")
statuses = ['Completed', 'Cancelled', 'Pending', 'Ongoing']
images = []

# Individual status charts + Cumulative chart
for status in statuses + ['Cumulative']:
    if status in pivot_chart.columns:
        fig, ax = plt.subplots(figsize=(16, 6))
        sns.lineplot(data=pivot_chart, x='order_date', y=status, ax=ax, label=status)
        ax.set_title(f'Daily Orders - {status}', fontsize=16)
        ax.set_xlabel('Order Date')
        ax.set_ylabel('Order Count')
        plt.xticks(rotation=45)
        plt.tight_layout()
        img_data = BytesIO()
        plt.savefig(img_data, format='png')
        plt.close()
        img_data.seek(0)
        images.append((f'Status_{status}', img_data))

# Combined chart: Completed vs Cumulative
if 'Completed' in pivot_chart.columns:
    fig, ax = plt.subplots(figsize=(16, 6))
    sns.lineplot(data=pivot_chart, x='order_date', y='Completed', ax=ax, label='Completed')
    sns.lineplot(data=pivot_chart, x='order_date', y='Cumulative', ax=ax, label='Cumulative')
    ax.set_title('Completed vs All Status (Cumulative)', fontsize=16)
    ax.set_xlabel('Order Date')
    ax.set_ylabel('Order Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    combo_img_data = BytesIO()
    plt.savefig(combo_img_data, format='png')
    plt.close()
    combo_img_data.seek(0)
    images.append(('Completed_vs_Cumulative', combo_img_data))

# -------------------------
# NEW: Step 6 - Monthly day-wise comparison charts
# -------------------------
# Prepare data: extract month and day from order_date
filtered_df['month'] = filtered_df['order_date'].dt.to_period('M')
filtered_df['day'] = filtered_df['order_date'].dt.day

# Completed orders day-wise per month
completed_df = filtered_df[filtered_df['Status'] == 'Completed']
completed_counts = (
    completed_df.groupby(['month', 'day'])
    .size()
    .unstack(level=0)
    .fillna(0)
    .sort_index()
)

# All orders (Completed + Cancelled + Pending + Ongoing) day-wise per month
valid_statuses = ['Completed', 'Cancelled', 'Pending', 'Ongoing']
all_orders_df = filtered_df[filtered_df['Status'].isin(valid_statuses)]
all_counts = (
    all_orders_df.groupby(['month', 'day'])
    .size()
    .unstack(level=0)
    .fillna(0)
    .sort_index()
)

# Plot Completed orders day-wise per month
fig, ax = plt.subplots(figsize=(16, 6))
completed_counts.plot(ax=ax, marker='o')
ax.set_title('Monthly Day-wise Completed Orders', fontsize=16)
ax.set_xlabel('Day of Month')
ax.set_ylabel('Order Count')
plt.xticks(range(1,32))
plt.legend(title='Month', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
completed_img_data = BytesIO()
plt.savefig(completed_img_data, format='png')
plt.close()
completed_img_data.seek(0)
images.append(('Monthly_Daywise_Completed', completed_img_data))

# Plot All orders day-wise per month
fig, ax = plt.subplots(figsize=(16, 6))
all_counts.plot(ax=ax, marker='o')
ax.set_title('Monthly Day-wise All Orders (Completed + Cancelled + Pending + Ongoing)', fontsize=16)
ax.set_xlabel('Day of Month')
ax.set_ylabel('Order Count')
plt.xticks(range(1,32))
plt.legend(title='Month', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
all_img_data = BytesIO()
plt.savefig(all_img_data, format='png')
plt.close()
all_img_data.seek(0)
images.append(('Monthly_Daywise_All_Orders', all_img_data))

print("📈 All charts (including monthly day-wise) created in memory.")

# -------------------------
# Step 7: Insert Charts to Excel (no local files)
# -------------------------
wb = load_workbook(output_file)
ws = wb.create_sheet('Visual_Charts')

row = 2
for name, img_data in images:
    img = ExcelImage(img_data)
    img.width = 1200
    img.height = 400
    ws.add_image(img, f'B{row}')
    row += 25  # vertical spacing between images

wb.save(output_file)
print(f"📊 Charts inserted into Excel: {output_file}")


✅ Pivot tables written to Excel.


C:\Users\Lenovo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\Lenovo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\Lenovo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\Lenovo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before opera

📈 All charts (including monthly day-wise) created in memory.
📊 Charts inserted into Excel: D:/DYD/AI-ML/Python-Scripts/Data/SV_SG_Pivot.xlsx
